# SKLEARN VARIATION

In [10]:
from sklearn.metrics import roc_auc_score
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

from aijack.attack.membership import ShadowMembershipInferenceAttack
from aijack.utils.utils import TorchClassifier, NumpyDataset

In [11]:

#load mnist dataset
from tensorflow.keras.datasets import mnist
#import shuffle from sklearn
from sklearn.utils import shuffle
#one hot encode from sklearn
from sklearn.preprocessing import OneHotEncoder



np.random.seed(42)
torch.manual_seed(42)



(X_train, y_train), (X_test, y_test) = mnist.load_data()

#shuffle the dataset
X_train, y_train = shuffle(X_train, y_train, random_state=42)
X_test, y_test = shuffle(X_test, y_test, random_state=42)

# downsample X_train and y_train to 1000 samples
X_train = X_train[:1000]
y_train = y_train[:1000]

X_train = X_train.reshape(-1, 28*28)
X_test = X_test.reshape(-1, 28*28)
# reshaping data
X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)

#normalizing data
X_train /= 255
X_test /= 255

#splitting train dataset into train and shadow dataset
X_train, X_shadow, y_train, y_shadow = train_test_split(
    X_train, y_train, test_size=1 / 2, random_state=42
)





# downsample X_test and y_test to 100 samples
X_test = X_test[:100]
y_test = y_test[:100]

#converting labels to int64
y_train = y_train.astype(np.long)
y_test = y_test.astype(np.long)
y_shadow = y_shadow.astype(np.long)


# We simulate the situation where the distribution of training dataset is different from the test/shadow datasets.
X_test = 0.5 * X_test + 0.5 * np.random.normal(size=(X_test.shape))
X_test=X_test.astype(np.float32)



/tmp/ipykernel_1977/3541508647.py:49: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_train = y_train.astype(np.long)
/tmp/ipykernel_1977/3541508647.py:50: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.com

Training the victim

In [12]:
# Train the victim

clf = SVC(probability=True)
clf.fit(X_train, y_train)
clf.score(X_train, y_train), clf.score(X_test, y_test)

(0.98, 0.17)

Training the attacker

In [13]:
# Train the attacker

shadow_models = [SVC(probability=True) for _ in range(2)]
attack_models = [SVC(probability=True) for _ in range(10)]

attacker = ShadowMembershipInferenceAttack(clf, shadow_models, attack_models)
attacker.fit(X_shadow, y_shadow)

Get the attack result of membership inference

In [14]:
# Get the attack result of membership inference
in_result = attacker.predict(clf.predict_proba(X_train), y_train)
out_result = attacker.predict(clf.predict_proba(X_test), y_test)

in_label = np.ones(in_result.shape[0])
out_label = np.zeros(out_result.shape[0])

accuracy_score(
    np.concatenate([in_label, out_label]), np.concatenate([in_result, out_result])
)

0.94

# PYTORCH VARIATION

In [15]:
class LM(nn.Module):
    def __init__(self):
        super(LM, self).__init__()
        self.lin1 = nn.Linear(28 * 28, 10)

    def forward(self, x):
        out = self.lin1(x)
        return out

Train the victim

In [16]:
# Train the victim
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

criterion = nn.CrossEntropyLoss()
net = LM().to(device)
optimizer = optim.Adam(net.parameters(), lr=0.001)
# You need to wrap the torch module with TorchClassifier
clf = TorchClassifier(
    net, criterion, optimizer, batch_size=64, epoch=100, device=device
)

clf.fit(X_train, y_train),clf.score(X_test, y_test)


(TorchClassifier(batch_size=64, criterion=CrossEntropyLoss(),
                 device=device(type='cpu'), epoch=100,
                 model=LM(
   (lin1): Linear(in_features=784, out_features=10, bias=True)
 ),
                 optimizer=Adam (
 Parameter Group 0
     amsgrad: False
     betas: (0.9, 0.999)
     capturable: False
     differentiable: False
     eps: 1e-08
     foreach: None
     fused: None
     lr: 0.001
     maximize: False
     weight_decay: 0
 )),
 0.52)

Train the attacker

In [17]:
# Train the attacker


def create_clf():
    _net = LM().to(device)
    _optimizer = optim.Adam(_net.parameters(), lr=0.001)
    return TorchClassifier(
        _net, criterion, _optimizer, batch_size=64, epoch=100, device=device
    )


shadow_models = [create_clf() for _ in range(2)]
attack_models = [SVC(probability=True) for _ in range(10)]

attacker = ShadowMembershipInferenceAttack(clf, shadow_models, attack_models)
attacker.fit(X_shadow, y_shadow)

In [18]:
# Get the attack result of membership inference
in_result = attacker.predict(clf.predict_proba(X_train), y_train)
out_result = attacker.predict(clf.predict_proba(X_test), y_test)

in_label = np.ones(in_result.shape[0])
out_label = np.zeros(out_result.shape[0])

att_acc=accuracy_score(
    np.concatenate([in_label, out_label]), np.concatenate([in_result, out_result])
)


att_pr=precision_score(
    np.concatenate([in_label, out_label]), np.concatenate([in_result, out_result])
)

att_r=recall_score(
    np.concatenate([in_label, out_label]), np.concatenate([in_result, out_result])
)

att_f1=f1_score(
    np.concatenate([in_label, out_label]), np.concatenate([in_result, out_result])
)

att_auc=roc_auc_score(
    np.concatenate([in_label, out_label]), np.concatenate([in_result, out_result])
)

print("Accuracy: ",att_acc)
print("Precision: ",att_pr)
print("Recall: ",att_r)
print("F1: ",att_f1)
print("AUC: ",att_auc)

Accuracy:  0.7383333333333333
Precision:  0.9193154034229829
Recall:  0.752
F1:  0.8272827282728272
AUC:  0.7109999999999999
